In [68]:
%load_ext autoreload
%autoreload 2

import requests
import sys
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests
from lxml import etree
import re

agent = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          'Accept-Language': 'pt-BR'}

latin_dict = \
            {'\\u00e1':'a', '\\u00e0':'a', '\\u00e2':'a', '\\u00e3':'a', '\\u00e4':'a', '\\u00c1':'A', '\\u00c0':'A', 
             '\\u00c2':'A', '\\u00c3':'A', '\\u00c4':'A', '\\u00e9':'e', '\\u00e8':'e', '\\u00ea':'e', '\\u00ea':'e', 
             '\\u00c9':'E', '\\u00c8':'E', '\\u00ca':'E', '\\u00cb':'E', '\\u00ed':'i', '\\u00ec':'i', '\\u00ee':'i', 
             '\\u00ef':'i', '\\u00cd':'I', '\\u00cc':'I', '\\u00ce':'I', '\\u00cf':'I', '\\u00f3':'o', '\\u00f2':'o', 
             '\\u00f4':'o', '\\u00f5':'o', '\\u00f6':'o', '\\u00d3':'O', '\\u00d2':'O', '\\u00d4':'O', '\\u00d5':'O', 
             '\\u00d6':'O', '\\u00fa':'u', '\\u00f9':'u', '\\u00fb':'u', '\\u00fc':'u', '\\u00da':'U', '\\u00d9':'U', 
             '\\u00db':'U', '\\u00e7':'c', '\\u00c7':'C', '\\u00f1':'n', '\\u00d1':'N', '\\u0026':'E', '\\u0027':'\'',
             'á':'a', 'à':'a', 'â':'a', 'ã':'a', 'ä':'a', 'Á':'A', 'À':'A', 'Â':'A', 'Ã':'A', 'Ä':'A', 'é':'e', 'è':'e', 
             'ê':'e', 'ê':'e', 'É':'E', 'È':'E', 'Ê':'E', 'Ë':'E', 'í':'i', 'ì':'i', 'î':'i', 'ï':'i', 'Í':'I', 'Ì':'I', 
             'Î':'I', 'Ï':'I', 'ó':'o', 'ò':'o', 'ô':'o', 'õ':'o', 'ö':'o', 'Ó':'O', 'Ò':'O', 'Ô':'O', 'Õ':'O', 'Ö':'O', 
             'ú':'u', 'ù':'u', 'û':'u', 'ü':'u', 'Ú':'U', 'Ù':'U', 'Û':'U', 'ç':'c', 'Ç':'C', 'ñ':'n', 'Ñ':'N', '&':'E'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generic Functions

In [4]:
def levenshtein(word1, word2):
    """
    https://medium.com/@yash_agarwal2/soundex-and-levenshtein-distance-in-python-8b4b56542e9e
    https://en.wikipedia.org/wiki/Levenshtein_distance
    :param word1:
    :param word2:
    :return:
    """
    word2 = word2.lower()
    word1 = word1.lower()
    matrix = [[0 for x in range(len(word2) + 1)] for x in range(len(word1) + 1)]

    for x in range(len(word1) + 1):
        matrix[x][0] = x
    for y in range(len(word2) + 1):
        matrix[0][y] = y

    for x in range(1, len(word1) + 1):
        for y in range(1, len(word2) + 1):
            if word1[x - 1] == word2[y - 1]:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1],
                    matrix[x][y - 1] + 1
                )
            else:
                matrix[x][y] = min(
                    matrix[x - 1][y] + 1,
                    matrix[x - 1][y - 1] + 1,
                    matrix[x][y - 1] + 1
                )

    return matrix[len(word1)][len(word2)]

def nomr_l(word1, word2):
    return levenshtein(word1, word2) / max(len(word1), len(word2))

def html_to_np(table):
    result = []
    all_rows = table.find_all('tr')
    for row in all_rows:
        result.append([])
        all_cols = row.find_all('td')
        for col in all_cols:
            col_text = [s.strip(' ') for s in col.find_all(text=True)]
            col_text  = ''.join(col_text) # it should not change anything
            result[-1].append(col_text)
    return np.array(result)


def np_to_dict(table, fields_dict):
    new_dict = fields_dict.copy()
    for cols in table:
        if len(cols) != 2:
            continue
        for key in fields_dict.keys():
            if nomr_l(key, cols[0]) <= 0.25:
                new_dict[key] = cols[1].replace("\n",'')
            #print(nomr_l(key, cols[0]), key, cols[0])
    return new_dict

## Amazon

In [66]:
def get_fields_amazon(html_page):
    fields_amazon = {
            'Preço': '',
            'Sistema operacional': '',
            'RAM': '',
            'Capacidade de armazenamento digital': '',
            'Modelo': '',
            'Cor': '',  
            'Tamanho da tela':'',
            'Avaliações de clientes': ''
           }
    
    parser = etree.HTMLParser()
    XPATH_TABLE1 = """//*[@id="prodDetails"]/div[2]/div[1]/div/div[2]/div/div/table/tbody/tr"""
    XPATH_TABLE2 = """//*[@id="prodDetails"]/div[2]/div[2]/div/div[2]/div/div/table/tbody/tr"""
    XPATH_AVAL = """//*[@id="prodDetails"]/div[2]/div[2]/div/div[2]/div/div/table/tbody/tr[10]/td[2]/span/span[3]/a/text()"""
    PRICE = """//*[@id="price"]/table/tr/td/span[1]/text()"""
    tree = etree.parse(html_page, parser)
    RAW_TABLE1 = tree.xpath(XPATH_TABLE1)
    RAW_TABLE2 = tree.xpath(XPATH_TABLE2)
    RAW_AVAL = tree.xpath(XPATH_AVAL)
    RAW_PRICE = tree.xpath(PRICE)
    
    try:
        fields_amazon['Preço'] = RAW_PRICE[0]
    except:
        pass
    
    try:
        fields_amazon = parse_table_amazon(fields_amazon, RAW_TABLE1)
        fields_amazon = parse_table_amazon(fields_amazon, RAW_TABLE2)
    except:
        pass
    
    try:
        fields_amazon['Avaliações de clientes'] = RAW_AVAL[0].strip()
    except:
        pass
                
    return fields_amazon

def parse_table_amazon(fields, table):
    try:
        for row in table:
            for key in fields:
                att = row[0].text.replace(" ", "")
                loss = nomr_l(key, att)
                if(loss < 0.3):
                    value = row[1].text
                    fields[key] = value
    except:
        pass
    
    return fields

In [67]:
# testing
# get_fields_amazon('./amazon.html')

{'Preço': 'R$1.039,90',
 'Sistema operacional': 'Android 8.1 Oreo',
 'RAM': '4 GB',
 'Capacidade de armazenamento digital': '256 GB',
 'Modelo': 'M1808D2TGN-64',
 'Cor': 'Preto',
 'Tamanho da tela': '6.26 polegadas',
 'Avaliações de clientes': '988 avaliações de clientes'}

## Magazine Luiza

In [46]:
def get_fields_luiza(html_page):

    fields_luiza = {
            'Preço': '',
            'Sistema operacional': '',
            'Memória RAM': '',
            'Memória': '',
            'Modelo': '',
            'Cor': '',  
            'Tamanho da tela':'',
            'Avaliações de clientes': '',
            'Marca' : ''
           }
    
    parser = etree.HTMLParser()
    XPATH_TABLE1 = """//*[@id="anchor-description"]/div/table[1]/tr/td/table/tr"""
    XPATH_TABLE2 = """//*[@id="anchor-description"]/div/table[2]/tr/td/table/tr"""
    PRICE = """/html/body/div[3]/div[5]/div[1]/div[4]/div[2]/div[4]/div/div/div/span"""
    tree = etree.parse(html_page, parser)
    RAW_TABLE1 = tree.xpath(XPATH_TABLE1)
    RAW_TABLE2 = tree.xpath(XPATH_TABLE2)
    RAW_PRICE = tree.xpath(PRICE)
    
    if(len(RAW_PRICE) >= 2):
        fields_luiza["Preço"] = "{} {}".format(RAW_PRICE[0].text, RAW_PRICE[1].text) 
    
    
    fields_luiza = parse_table_luiza(fields_luiza, RAW_TABLE1)
    fields_luiza = parse_table_luiza(fields_luiza, RAW_TABLE2)
                
    return fields_luiza

def parse_table_luiza(fields, table):
    try:
        for row in table:
            for key in fields:
                att = row[0].text.replace(" ", "")
                loss = nomr_l(key, att)
                if(loss < 0.3):
                    value = row[1].text
                    fields[key] = value
    except:
        pass
    return fields

In [47]:
# testing
# get_fields_luiza('./luiza.html')

{'Preço': 'R$ 4.999,00',
 'Sistema operacional': '  Android  ',
 'Memória RAM': '  8GB  ',
 'Memória': '  128GB  ',
 'Modelo': '  Galaxy S10  ',
 'Cor': '  Azul  ',
 'Tamanho da tela': '  6.1 polegadas  ',
 'Avaliações de clientes': '',
 'Marca': '  Samsung  '}

# IByte

In [42]:
def get_fields_ibyte(html_page):
    fields_ibyte = {
            'Preço': '',
            'Sistema operacional': '',
            'Memória RAM(GB)': '',
            'Memória Total Interna (GB)': '',
            'Modelo': '',
            'Cor': '',  
            'Tamanho (Tela Principal)':'',
            'Avaliações de clientes': '',
            'Marca' : ''
           }
    
    parser = etree.HTMLParser()
    XPATH_DD = """//*[@id="descricao"]/table/tbody/tr"""
    PRICE = """//*[@id="product_addtocart_form"]/div[3]/div[1]/div/div/p[2]"""
    tree = etree.parse(html_page, parser)
    RAW_DD = tree.xpath(XPATH_DD)
    RAW_PRICE = tree.xpath(PRICE)
    
    try:
        fields_ibyte['Preço'] = RAW_PRICE[0][1].text.replace("\n", "").replace(" ", "")
    except:
        pass
    
    try:
        for row in RAW_DD:
            if(len(row) == 3):
                att = row[1][0].text
                value  = row[2].text
                for key in fields_ibyte.keys():
                    loss = nomr_l(key, att)
                    if loss < 0.2:
                        fields_ibyte[key] = value
    except:
        pass

    return fields_ibyte

In [43]:
# testing
# get_fields_ibyte('./ibyte.html')

{'Preço': 'R$2.999,00',
 'Sistema operacional': 'Android',
 'Memória RAM(GB)': '4 GB',
 'Memória Total Interna (GB)': '128 GB*',
 'Modelo': '',
 'Cor': '',
 'Tamanho (Tela Principal)': '146.5mm (5.8" retângulo cheio) / 143.3mm (5.6" cantos arredondados)',
 'Avaliações de clientes': '',
 'Marca': ''}

# Kabum

In [40]:
def get_fields_kabum(html_page):
    fields_kabum = {
            'Preço': '',
            'Sistema operacional': '',
            'Memória RAM': '',
            'Memória Interna total compartilhada': '',
            'Modelo': '',
            'Cor': '',  
            'Tamanho do Display':'',
            'Avaliações de clientes': '',
            'Marca' : ''
           }
    
    parser = etree.HTMLParser()
    XPATH_DD = """//*[@id="pag-detalhes"]/div/div[6]/div[2]/p"""
    PRICE = """//*[@id="pag-detalhes"]/div/div[2]/div[2]/div[2]/div[1]/span/span/span/strong"""
    AVAL = """//*[@id="pag-detalhes"]/div/div[2]/div[1]/div[1]/div[1]/div[2]/table"""
    tree = etree.parse(html_page, parser)
    RAW_DD = tree.xpath(XPATH_DD)
    RAW_PRICE = tree.xpath(PRICE)
    RAW_AVAL = tree.xpath(AVAL)
    
    try:
        fields_kabum['Avaliações de clientes'] = RAW_AVAL[0][0][0].text.split("(")[1].split(")")[0]
    except:
        pass
    
    try:
        fields_kabum['Preço'] = RAW_PRICE[0].text
    except:
        pass
    try:
        
        for p in RAW_DD:
            if(p.text != None):
                info = p.text.split(":")
                if len(info) == 2:
                    attrib, value = info
                    for key in fields_kabum.keys():
                        loss = nomr_l(key, attrib)
                        if loss < 0.4:
                            fields_kabum[key] = value
    except:
        pass
    
    return fields_kabum

In [41]:
# testing
# get_fields_kabum('./kabum.html')

{'Preço': 'R$ 674,10 ',
 'Sistema operacional': ' Android 9.0',
 'Memória RAM': ' 2GB',
 'Memória Interna total compartilhada': ' 32GB*',
 'Modelo': ' SM-A105M/32DL',
 'Cor': '',
 'Tamanho do Display': ' 6.2"*',
 'Avaliações de clientes': '10',
 'Marca': ' Samsung'}

# Avenida

In [25]:
def get_fields_avenida(html_page):
    fields_avenida = {'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Memória Interna': '',
        'Modelo': '',
        'Cor': '',  
        'Tamanho do Display':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }

    parser = etree.HTMLParser()
    XPATH_DD = """//*[@id="caracteristicas"]/table"""
    PRICE = """//*[@id="product-content"]/div[2]/div[1]/div[2]/div/div[3]/div/p/em/strong"""
    tree = etree.parse(html_page, parser)
    RAW_DD = tree.xpath(XPATH_DD)
    RAW_DD = RAW_DD[0]
    RAW_PRICE = tree.xpath(PRICE)
    RAW_PRICE = RAW_PRICE
    
    try:
        fields_avenida['Preço'] = RAW_PRICE[1].text
    except:
        pass
    try:
        for key in fields_avenida.keys():
            for element in RAW_DD:
                loss = nomr_l(element[0].text, key)
                if loss < 0.4:
                    fields_avenida[key] = element[1].text
    except:
        pass

    return fields_avenida

In [163]:
# testing
# get_fields_avenida('./avenida.html')

{'Preço': 'R$ 699,99',
 'Sistema operacional': '',
 'Memória RAM': '1GB',
 'Memória Interna': '8GB',
 'Modelo': 'Redmi Go',
 'Cor': 'Preto',
 'Tamanho do Display': '5"',
 'Avaliações de clientes': '',
 'Marca': 'Xiaomi'}

## Cissamagazine

In [33]:
def get_fields_cissamagazine(html_page):
    fields_cissamagazine = {
            'Preço': '',
            'Sistema operacional': '',
            'Memória RAM': '',
            'Memória Interna': '',
            'Modelo': '',
            'Cor': '',  
            'Tamanho da tela':'',
            'Avaliações de clientes': '',
            'Marca' : ''
           }

    parser = etree.HTMLParser()
    XPATH_DD = r"""//*[@id="caracteristicas"]/div[2]/div/dl/span/dd/text()"""
    XPATH_DT = r"""//*[@id="caracteristicas"]/div[2]/div/dl/span/dt/text()"""
    PRICE = """//*[@id="content-price-product"]/div[1]/div[2]/div/span[1]/span[1]/text()"""
    tree = etree.parse(html_page, parser)
    RAW_DD = tree.xpath(XPATH_DD)
    RAW_DT = tree.xpath(XPATH_DT)
    RAW_PRICE = tree.xpath(PRICE)
    
    for p in RAW_PRICE:
        if len(p):
            price = (re.findall("R\$ \d*.\d{1,5}\,\d{1,3}", str(p)))
            if len(price):
                fields_cissamagazine["Preço"] = price[0]


    try:
        for p in zip(RAW_DT, RAW_DD):
            for key in fields_cissamagazine.keys():
                loss = nomr_l(p[0], key)
                if loss < 0.2:
                    fields_cissamagazine[key] = p[1]
    except:
        pass

    return fields_cissamagazine

In [25]:
# testing
# get_fields_cissamagazine('./cissamagazine.html')

{'Preço': 'R$ 1.838,50',
 'Sistema operacional': '',
 'Memória RAM': '6 GB',
 'Memória Interna': '128 GB',
 'Modelo': ' Wi-Fi 802.11 a/b/g/n/ac; ',
 'Cor': '',
 'Tamanho da tela': '6.39"',
 'Avaliações de clientes': '',
 'Marca': 'Bluetooth 5.0; '}

## Colombo

In [34]:
# html_page = string with the path to the html page
def get_fields_colombo(html_page):
    fields_colombo = {
        'Preço': '',
        'Sistema operacional': '',
        'Memória RAM': '',
        'Memória Interna': '',
        'Modelo': '',
        'Cor': '',  
        'Resolução da tela':'',
        'Avaliações de clientes': '',
        'Marca' : ''
       }
    
    parser = etree.HTMLParser()
    XPATH_DD = r"""//*[@id="produto-caracteristicas"]/div/div[2]/div/div/span/text()"""
    PRICE = """//*[@id="dados-produto-disponivel"]/div[2]/span[2]/span/text()"""
    tree = etree.parse(html_page, parser)
    RAW_DD = tree.xpath(XPATH_DD)
    RAW_PRICE = tree.xpath(PRICE)
    
    for idx, p in enumerate(RAW_PRICE):
        p = p.strip()
        fd = re.findall("\d*.\d{1,5}\,\d{1,3}", str(p))
        if len(fd):
            fields_colombo["Preço"] = "R$ " + fd[0]

    try:
        
        for idx, p in enumerate(RAW_DD):
            p = p.strip()
            if idx % 2 == 0:
                for key in fields_colombo.keys():
                    loss = nomr_l(p, key)

                    if loss <= 0.25:
                        fields_colombo[key] = RAW_DD[idx+1].strip().split('\n')[0]
    except:
        pass
                    
    return fields_colombo

In [35]:
# testing
# get_fields_colombo('./colombo.html')

{'Preço': 'R$ 858,00',
 'Sistema operacional': 'Android 7.0',
 'Memória RAM': '2 GB',
 'Memória Interna': '32 GB',
 'Modelo': 'ZenFone Max M1',
 'Cor': 'Preto',
 'Resolução da tela': '1280 x 720',
 'Avaliações de clientes': '',
 'Marca': ''}

# Ricardo Eletro

In [36]:
def get_fields_ricardo(html_page):
    fields_ricardo = {
            'Preço': '',
            'Sistema operacional': '',
            'Memória RAM': '',
            'Armazenamento Interno': '',
            'Modelo': '',
            'Cor': '',  
            'Resolução da tela':'',
            'Avaliações de clientes': '',
            'Marca' : ''
           }


    parser = etree.HTMLParser()
    XPATH_DD = r"""//*[@id="aba-caracteristicas"]/div[1]/table[1]"""
    PRICE = r"""//*[@id="ProdutoDetalhesPrecoComprarAgoraPrecoDePreco"]/text()"""
    tree = etree.parse(html_page, parser)
    RAW_DD = tree.xpath(XPATH_DD)
    RAW_PRICE = tree.xpath(PRICE)
    
    for p in RAW_PRICE:
        fields_ricardo["Preço"] = "R$ "+p
    try:
        
        for idx, p in enumerate(RAW_DD):
            tr = p.findall("tr")
            for y in range(len(tr)):
                propesed_key, propesed_value = tr[y][0], tr[y][1]
                for key in fields_ricardo.keys():

                    loss = nomr_l(propesed_key.text, key)
                    if loss <= 0.35:
                        fields_ricardo[key] = propesed_value.text
    except:
        pass
    return fields_ricardo

In [39]:
# testing
# get_fields_ricardo('./ricardoeletro.html')

{'Preço': 'R$ 735,42',
 'Sistema operacional': 'ANDROID 8.1 ',
 'Memória RAM': '2 GB ',
 'Armazenamento Interno': '16 GB ',
 'Modelo': '',
 'Cor': 'Preto ',
 'Resolução da tela': 'HD+ (1440X720) ',
 'Avaliações de clientes': '',
 'Marca': ''}